# Using autodiff

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup autodiff and Eigen](#toc1_)    
- 2. [Compile test file](#toc2_)    
- 3. [Run test file](#toc3_)    
- 4. [Clean up](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

This notebook shows how to use [autodiff](https://autodiff.github.io/) for **automatic differentiation of C++ functions**.

We consider the following *multi-variable* and *vector-valued* function:

$$ 
F:\mathbb{R}^{3}\rightarrow\mathbb{R}^{2}:\,F(\boldsymbol{x}) = F(x_{1},x_{2},x_{2})=\left[\begin{array}{c}
y_{1}\\
y_{2}
\end{array}\right]=\left[\begin{array}{c}
f_{1}(x_{1},x_{2},x_{3})\\
f_{2}(x_{1},x_{2},x_{3})
\end{array}\right]=\left[\begin{array}{c}
\alpha x_{1}+\exp x_{3}\\
\beta x_{2}
\end{array}\right]
$$

In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import numpy as np
from types import SimpleNamespace

from EconModel import cpptools

## 1. <a id='toc1_'></a>[Setup autodiff and Eigen](#toc0_)

Download files and create **cppfuncs/autodiff**:

In [2]:
cpptools.setup_autodiff(do_print=True)

autodiff succesfully installed


Download files and create **cppfuncs/Eigen**:

In [3]:
cpptools.setup_Eigen(do_print=True)

Eigen succesfully installed


## 2. <a id='toc2_'></a>[Compile test file](#toc0_)

**Setup namespace:**

In [4]:
par = SimpleNamespace()

par.alpha = 0.5
par.beta = 2.0

par.Nx = 3
par.Ny = 2

par.x = np.zeros(par.Nx)
par.y = np.zeros(par.Ny)
par.Jx = np.zeros((par.Ny,par.Nx))

**Compile:**

In [5]:
filename = 'cppfuncs/example_autodiff.cpp'
structsmap = {'par_struct':par}
flags = '/LD /EHsc /Ox /std:c++17 /Icppfuncs/' # requires c++17 and references are relative to cppfuncs
options = {'compiler':'vs','flags':flags}

example_autodiff = cpptools.link_to_cpp(filename,structsmap=structsmap,options=options)

## 3. <a id='toc3_'></a>[Run test file](#toc0_)

**Set x for evaluation:**

In [6]:
par.x[:] = np.array([1.0,1.0,1.0])

**Run:**

In [7]:
example_autodiff.test(par)

**Output:**

In [8]:
print(par.y)

[3.21828183 2.        ]


**Jacobian:**

$$ 
\boldsymbol{J}_{\boldsymbol{x}}=\left[\begin{array}{ccc}
\frac{\partial f_{1}}{\partial x_{1}} & \frac{\partial f_{1}}{\partial x_{2}} & \frac{\partial f_{1}}{\partial x_{3}}\\
\frac{\partial f_{2}}{\partial x_{1}} & \frac{\partial f_{2}}{\partial x_{2}} & \frac{\partial f_{2}}{\partial x_{3}}
\end{array}\right]
$$

In [9]:
print(par.Jx)

[[0.5        0.         2.71828183]
 [0.         2.         0.        ]]


## 4. <a id='toc4_'></a>[Clean up](#toc0_)

In [10]:
example_autodiff.clean_up()

In [11]:
def remove(filename):
    if os.path.exists(filename): os.remove(filename)
    
remove(f'cppfuncs/autodiff-main.zip')
remove(f'cppfuncs/Eigen-main.zip')
shutil.rmtree(f'cppfuncs/autodiff/')
shutil.rmtree(f'cppfuncs/Eigen/')